In [1]:
from MCTSOpt import Tree
from MCTSOpt.SelectionRule_Stocastic import UBStocastic
from MCTSOpt.ParameterOpt import LogisticSearch
from time import time
import numpy as np

In [2]:
from Model import getexploremodel
model = getexploremodel()

nparameters = 0
for layer in model.trainable_weights:
    print(layer.shape)
    nparameters += layer.shape.num_elements()
print("Number of parameters: ", nparameters)



Using TensorFlow backend


2023-10-17 16:34:57.745382: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 16:34:57.803641: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 16:34:57.804954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 16:34:58.757742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(1, 32)
(32,)
(32, 64)
(64,)
(64, 64)
(64,)
(64, 32)
(32,)
(32, 1)
(1,)
Number of parameters:  8449


/usr/local/lib/python3.10/dist-packages/keras_core/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
from ScoringFunction import MCTS_MacroScore
deadzonewidth = 0.25
objfunc = MCTS_MacroScore(model, deadzone=deadzonewidth)

Using a Deadzone of width: 0.25
Deadzone low/high: -0.125, 0.125
X Bounds:  -3 3
Y Bounds:  -3 3
X Bounds:  -30 30
Y Bounds:  -30 30
X Bounds:  -2.048 2.048
Y Bounds:  -2.048 2.048
X Bounds:  -3.141592653589793 3.141592653589793
Y Bounds:  -3.141592653589793 3.141592653589793
X Bounds:  -600 600
Y Bounds:  -600 600
X Bounds:  -50 50
Y Bounds:  -50 50
X Bounds:  -1.28 1.28
Y Bounds:  -1.28 1.28
X Bounds:  -5.12 5.12
Y Bounds:  -5.12 5.12
X Bounds:  -65.536 65.536
Y Bounds:  -65.536 65.536
X Bounds:  -100 100
Y Bounds:  -100 100
X Bounds:  -10 10
Y Bounds:  -10 10
X Bounds:  -500 500
Y Bounds:  -500 500
X Bounds:  -6 6
Y Bounds:  -6 6
X Bounds:  -10 10
Y Bounds:  -10 10
X Bounds:  0 3.141592653589793
Y Bounds:  0 3.141592653589793
X Bounds:  -7 7
Y Bounds:  -7 7
X Bounds:  -512 512
Y Bounds:  -512 512
Object Type: DeadZone, Depth: 0
Object Type: Function_Ackley, Depth: 1
Object Type: Function_AckleyTest, Depth: 2
Object Type: Function_Rosenbrock, Depth: 3
Object Type: Function_Fletcher, 

In [4]:
depthscale = [1.0, 0.7, 0.5, 0.3, 0.2, 0.1]
depthlimit = len(depthscale)+5

startset = [0.0]*nparameters
lbounds = [-1.5-deadzonewidth]*nparameters
ubounds = [ 1.5+deadzonewidth]*nparameters

startset = np.array(startset)
lbounds = np.array(lbounds)
ubounds = np.array(ubounds)



indata = LogisticSearch(parameters=startset, lossfunction=objfunc,  lbounds=lbounds, ubounds=ubounds, depthscale=depthscale)

In [5]:




#---Tree Main Run loop---
#Critical Parameters to Set
tree = Tree(seeddata=indata, 
        playouts=20, 
        selectfunction=UBStocastic, 
        headexpansion=30,
        verbose=True
        )
tree.expand(nExpansions=1)
tree.setconstant(5.37e2)
starttime = time()
for iLoop in range(1,150):
    print("Loop Number: %s"%(iLoop))
    tree.playexpand(nExpansions=1, depthlimit=depthlimit)
    tree.autoscaleconstant(scaleboost=2.0)
    tree.simulate(nSimulations=1)
    curtime = time()
    curmin = objfunc.bestscore
    print("Search Duration: %s, Best Score:%s"%(curtime-starttime, curmin))

Starting Chain, Depth: 0


TypeError: unsupported operand type(s) for *=: 'float' and 'dict'